# Implementando uma RBM para o Estado Fundamental do H₂

Este notebook demonstra como usar uma **Restricted Boltzmann Machine (RBM)** como uma *Função de Onda de Ansatz* (Neural Quantum State) para encontrar a energia do estado fundamental da molécula $H_2$.

Usaremos:
1.  **PySCF:** Para definir a molécula e obter as integrais (o Hamiltoniano).
2.  **OpenFermion:** Para converter o Hamiltoniano em um formato de "qubit" (operadores de Pauli) que o NetKet possa usar.
3.  **NetKet:** Para construir a RBM, rodar a otimização Variational Monte Carlo (VMC) e encontrar a energia.

## Passo 1: Instalação das Bibliotecas

In [1]:
# Rode no terminal ou descomente a linha abaixo no notebook:
!pip install pyscf netket openfermion openfermion-pyscf

ERROR: Could not find a version that satisfies the requirement openfermion-pyscf (from versions: none)
ERROR: No matching distribution found for openfermion-pyscf


In [ ]:
import pyscf
from pyscf import gto, scf, fci
import openfermion
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermionpyscf import run_pyscf

import netket as nk
import jax
import numpy as np
import jax.numpy as jnp
from netket.operator.spin import sigmax, sigmaz, sigmay
from netket.operator import PauliStrings

# Configurar NetKet para usar 64 bits para precisão
from jax.config import config
config.update("jax_enable_x64", True)

## Passo 2: Obter o Hamiltoniano (com PySCF)

Primeiro, definimos nossa molécula H₂ na base `sto-3g`, exatamente como no notebook FCI. Isso nos dá 2 elétrons em 4 orbitais de spin. 

Usaremos o OpenFermion para extrair o Hamiltoniano e convertê-lo em um formato de "qubit" (Transformação de Jordan-Wigner). Um sistema de 4 orbitais de spin se torna um sistema de 4 qubits.

In [ ]:
# 1. Definir a molécula no PySCF
mol = gto.M(
    atom='H 0 0 0; H 0 0 0.74',
    basis='sto-3g',
    spin=0
)

# 2. Rodar o PySCF e obter o Hamiltoniano Molecular
# (Usamos a ferramenta 'run_pyscf' do OpenFermion)
pyscf_analysis = run_pyscf(mol, run_scf=True, run_fci=True)

# 3. Obter o Hamiltoniano molecular no formato OpenFermion
H_molecular = pyscf_analysis.molecule.get_molecular_hamiltonian()

# 4. Converter para Operadores Fermiônicos
H_fermionic = get_fermion_operator(H_molecular)

# 5. Fazer a Transformação de Jordan-Wigner (Mapear para Qubits/Spins)
H_qubit = jordan_wigner(H_fermionic)

# Energia FCI (nosso "benchmark" exato)
E_fci = pyscf_analysis.fci_energy
print(f"Hamiltoniano gerado. Energia FCI (Exata) de referência: {E_fci:.8f} Ha")

## Passo 3: Definir o Problema no NetKet

Agora, passamos o problema para o NetKet.

In [ ]:
# 1. Definir o Espaço de Hilbert (o "tabuleiro")
# Temos 4 orbitais de spin -> 4 qubits (ou 4 spins 1/2)
# O NetKet usa a convenção de spin: Ocupado=Spin Down (-1), Vazio=Spin Up (+1)
N_orbitals = 4
hi = nk.hilbert.Spin(s=1/2, N=N_orbitals)

# 2. Definir o Hamiltoniano no NetKet
# O NetKet não entende o H_qubit do OpenFermion diretamente, precisamos converter
# os termos de Pauli (X, Y, Z) para o formato do NetKet.

# O Hamiltoniano 'H_qubit' é uma soma de strings de Pauli, ex: 0.1 [Z0 Z1] + 0.2 [X0 Y1 ...]
operators = []
coeffs = []

for pauli_term, coeff in H_qubit.terms.items():
    if not pauli_term:  # Termo de identidade (constante)
        operators.append(nk.operator.LocalOperator(hi, 1.0))
        coeffs.append(jnp.real(coeff))
        continue
        
    op_list = [[] for _ in range(N_orbitals)]
    for qubit_index, op_str in pauli_term:
        if op_str == 'X':
            op_list[qubit_index] = [sigmax(hi, qubit_index)]
        elif op_str == 'Y':
            op_list[qubit_index] = [sigmay(hi, qubit_index)]
        elif op_str == 'Z':
            op_list[qubit_index] = [sigmaz(hi, qubit_index)]

    # O OpenFermion usa coeficientes complexos, pegamos a parte real
    operators.append(PauliStrings(op_list))
    coeffs.append(jnp.real(coeff))

# Hamiltoniano NetKet final
H_netket = nk.operator.LocalOperator(operators, coeffs)

print(f"Espaço de Hilbert e Hamiltoniano construídos para NetKet.")

## Passo 4: Definir o Modelo (a RBM)

Definimos a arquitetura da RBM. A RBM padrão do NetKet funciona bem. O hiperparâmetro `alpha` controla quão "complexa" (quantos neurônios ocultos) a RBM é. `alpha=1` significa 1 neurônio oculto por neurônio visível.

In [ ]:
# 1. Definir o modelo RBM
# alpha=1 significa (1 * N_orbitals) = 4 neurônios ocultos
model = nk.models.RBM(alpha=1, param_dtype=jnp.float64)

# 2. Definir o Amostrador MCMC
# Usaremos o amostrador de Metropolis para spins
sampler = nk.sampler.MetropolisExchange(hilbert=hi, n_chains=16)

# 3. Definir o Otimizador
optimizer = nk.optimizer.Sgd(learning_rate=0.01)

# 4. Definir o método de Gradiente
# Usaremos Stochastic Reconfiguration (SR), que é um "gradiente natural"
sr = nk.driver.SR(diag_shift=0.1)

# 5. Criar o estado VMC (Variational Monte Carlo)
vs = nk.vqs.MCState(sampler, model, n_samples=1024)

# 6. Criar o "Driver" (O laço de otimização VMC)
vmc = nk.driver.VMC(
    hamiltonian=H_netket,
    optimizer=optimizer,
    variational_state=vs,
    preconditioner=sr
)

## Passo 5: Rodar a Otimização

Treinamos a RBM para 300 épocas e salvamos a energia a cada passo.

In [ ]:
# Configurar o log para salvar a energia a cada iteração
log = nk.logging.JsonLog(f"rbm_h2_log")

print("Iniciando a otimização VMC da RBM...")
vmc.run(n_iter=300, out=log)
print("Otimização concluída.")

# A energia final otimizada está no estado variacional
E_rbm = vs.expect(H_netket)

## Passo 6: Análise dos Resultados

Comparamos a energia que a RBM encontrou com a energia exata do Full CI.

In [ ]:
print("--- Comparação de Resultados ---")
print(f"Energia Full CI (Exata):     {E_fci:.8f} Ha")
print(f"Energia RBM (NetKet, VMC): {E_rbm.mean:.8f} Ha (+/- {E_rbm.error_of_mean:.8f})")

diferenca = (E_rbm.mean - E_fci)
print(f"Diferença (Erro da RBM):     {diferenca:.8f} Ha")
print(f"(O valor da RBM deve ser sempre >= ao FCI, devido ao Princípio Variacional)")

In [ ]:
# Opcional: Plotar a convergência da energia
import matplotlib.pyplot as plt

data = log.data
iterations = data['Energy']['iter']
energy_mean = data['Energy']['Mean']

plt.figure(figsize=(10, 6))
plt.plot(iterations, energy_mean, label='Energia VMC (RBM)')
plt.axhline(E_fci, color='r', linestyle='--', label='Energia FCI (Exata)')
plt.xlabel('Iteração de Otimização')
plt.ylabel('Energia (Ha)')
plt.title('Convergência da Energia VMC para RBM (H₂ sto-3g)')
plt.legend()
plt.grid(True)
plt.show()